In [1]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from datasets import MNISTDataset
# from show_comp_graph import show_graph
from time import time
import numpy as np
import tensorflow as tf
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
FLAGS = None

In [2]:
from sklearn.model_selection import train_test_split
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

### checking up the data

In [5]:
mnist = MNISTDataset("./MNIST_IMAGES", batch_size=100,seed=int(time()))
## normalizing the values
mnist.train_data = mnist.train_data/256
mnist.test_data = mnist.test_data/256
## one hot encoding of the labels
onehot_encoder = OneHotEncoder(sparse=False)
mnist.train_labels = onehot_encoder.fit_transform(mnist.train_labels.reshape(-1, 1))
mnist.test_labels = onehot_encoder.fit_transform(mnist.test_labels.reshape(-1, 1))

In [ ]:
## the training data
print("[",len(mnist.train_data),",",len(mnist.train_data[0]),"]")
print("[",len(mnist.train_labels),",",len(mnist.train_labels[0]),"]")
## the testing data
print("[",len(mnist.test_data),",",len(mnist.test_data[0]),"]")
print("[",len(mnist.test_labels),",",len(mnist.test_labels[0]),"]")
# ## the validation data
# print("[",len(mnist.validation.images),",",len(mnist.validation.images[0]),"]")
# print("[",len(mnist.validation.labels),",",len(mnist.validation.labels[0]),"]")

In [ ]:
# ## printing to peek at a flattened imageand its label
# print(mnist.train_data[0])
# print(mnist.train_labels[0])

### the global variable and the main function for doing everything basically

In [ ]:
FLAGS = None

In [ ]:
def main(_):
    # Import data
    global mnist
    
    # Create the model
    x = tf.placeholder(tf.float32, [None, 784])
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    
    y = tf.matmul(x, W) + b
    
    # one hidden layer //linear
    W1 = tf.Variable(tf.zeros([784, 10]))
    b1 = tf.Variable(tf.zeros([10]))
    y = tf.matmul(x, W1) + b1
    
    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])

#     # The cross-entropy
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
    cross_entropy = tf.reduce_mean(cross_entropy)
    print("entropy done")
    tf.summary.scalar("cost", cross_entropy)
    
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
#     train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    sess = tf.InteractiveSession()
    writer = tf.summary.FileWriter("/Tensorboard_Sem3", sess.graph) ## source file to store tensorboard files
    tf.global_variables_initializer().run()
    merged = tf.summary.merge_all()
    print("model creation done, training about to start")
    # Train
    for step in range(1000):
        batch_xs, batch_ys = mnist.next_batch()
#         print("step "+ str(step)+" running")
        stats, _ = sess.run([merged,train_step], feed_dict={x: batch_xs, y_: batch_ys})
        writer.add_summary(stats, step)
    print("training done")    
    # Test trained model
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    show_graph(tf.get_default_graph().as_graph_def())
    print("The accuracy:")
    print(sess.run(accuracy, feed_dict={x: mnist.test_data,
                                      y_: mnist.test_labels}))
    

In [ ]:
if __name__ == '__main__':
    tf.app.run(main=main)

In [ ]:
%tb